In [16]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("C://Users\ADMIN//Desktop//Data Analysis//datasets//german_credit_data.csv")
df = df.drop('Unnamed: 0', axis=1)
df.head()

<>:6: SyntaxWarning: invalid escape sequence '\A'
<>:6: SyntaxWarning: invalid escape sequence '\A'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24340\1513894082.py:6: SyntaxWarning: invalid escape sequence '\A'
  df = pd.read_csv("C://Users\ADMIN//Desktop//Data Analysis//datasets//german_credit_data.csv")


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,NaN,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,NaN,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car


In [17]:
df.isnull().sum()

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
dtype: int64

In [18]:
df.fillna(method='bfill', inplace=True)
df.isnull().sum()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24340\3836732734.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
dtype: int64

In [20]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Encode categorical variables (reuse existing le if already defined)
le = LabelEncoder()
categorical_cols = ['Sex', 'Housing', 'Saving accounts', 'Checking account', 'Purpose']
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

# Prepare features (no target needed)
X = df

# Train Isolation Forest model for anomaly detection (contamination=0.1 assumes ~10% anomalies are high-risk)
model = IsolationForest(random_state=42, contamination=0.1)
model.fit(X)

# Predict anomaly scores (negative scores indicate anomalies/higher risk)
anomaly_scores = model.decision_function(X)
df['Risk_Score'] = anomaly_scores

# Identify high-risk instances (e.g., top 10% as high risk)
threshold = df['Risk_Score'].quantile(0.1)  # Lower 10% are anomalies
df['Predicted_Risk'] = df['Risk_Score'].apply(lambda x: 'High' if x <= threshold else 'Low')

# Display results
print("Anomaly Scores (lower = higher risk):")
print(df[['Risk_Score', 'Predicted_Risk']].head(10))
print("\nValue counts of predicted risk:")
print(df['Predicted_Risk'].value_counts())


Anomaly Scores (lower = higher risk):
   Risk_Score Predicted_Risk
0    0.101386            Low
1    0.046909            Low
2    0.114284            Low
3    0.051022            Low
4    0.088323            Low
5   -0.025010           High
6    0.091094            Low
7    0.047015            Low
8    0.008116            Low
9    0.110400            Low

Value counts of predicted risk:
Predicted_Risk
Low     900
High    100
Name: count, dtype: int64


In [22]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

# Assuming true labels are available as 'True_Risk_Label' (synthetic or actual)
# Map to numeric for metrics
y_true = df['True_Risk_Label'].map({'High': 1, 'Low': 0})
y_pred = df['Predicted_Risk'].map({'High': 1, 'Low': 0})
y_score = -df['Risk_Score']  # Negative because lower score indicates higher risk

# Compute evaluation metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_score)

print("Evaluation Metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {auc:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix (High Risk ~ Positive):")
print("              Predicted")
print("True     Low    High")
print(f"Low      {cm[0,0]}    {cm[0,1]}")
print(f"High     {cm[1,0]}    {cm[1,1]}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Low', 'High']))

Evaluation Metrics:
Precision: 0.4700
Recall: 0.0913
F1-Score: 0.1528
ROC-AUC: 0.4710

Confusion Matrix (High Risk ~ Positive):
              Predicted
True     Low    High
Low      432    53
High     468    47

Classification Report:
              precision    recall  f1-score   support

         Low       0.48      0.89      0.62       485
        High       0.47      0.09      0.15       515

    accuracy                           0.48      1000
   macro avg       0.47      0.49      0.39      1000
weighted avg       0.47      0.48      0.38      1000

